In [10]:
#Install essential pip
!python -m pip install --upgrade pip
!pip install geocoder==1.5.0
!pip install geopy
!pip install folium


Requirement already up-to-date: pip in c:\users\v\anaconda3\lib\site-packages (19.3.1)


In [93]:
#Importing Essential libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [21]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
#Appending Similar Neighbourhoods
tb = tables[0].groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)
t2=pd.DataFrame(tb)
#Dropping Not Assigned Neighbourhoods
t3=t2.reset_index()
t3=t3[t3.Borough != 'Not assigned']
t3 = t3.sort_values(by=['Postcode','Borough'])
t3.reset_index(inplace=True)
t3.drop('index',axis=1,inplace=True)
tables2 = pd.read_csv('https://cocl.us/Geospatial_data',header=0)
tables2.drop('Postal Code',axis=1,inplace=True)
tab3 = t3.join(tables2)
tab3.head()



,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [37]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.653963, -79.387207.


In [38]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tab3['Latitude'], tab3['Longitude'], tab3['Borough'], tab3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [41]:
# Exploring a particular Borough
EastYork_data = tab3[tab3['Borough'] == 'East York'].reset_index(drop=True)
EastYork_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
1,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372
4,M4J,East York,East Toronto,43.685347,-79.338106


In [45]:
address = 'East York, Toronto.'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East York are 43.699971, -79.3325199626159.


In [48]:
# create map of East York using latitude and longitude values
map_eastyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(EastYork_data['Latitude'], EastYork_data['Longitude'], EastYork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_eastyork)  
    
map_eastyork

 <font color='red'>CLIENT_ID AND CLIENT_SECRET HAVE BEEN REMOVED DUE TO PRIVACY!</font> 

In [51]:
EastYork_data.loc[0,'Neighborhood']

'Woodbine Gardens,Parkview Hill'

In [52]:
neighborhood_latitude = EastYork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = EastYork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = EastYork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woodbine Gardens,Parkview Hill are 43.7063972, -79.309937.


In [104]:
# Top venues in Woodbine Gardens and Parkview Hills within 500 meters
LIMIT = 100
radius = 500
#CLIENT_ID AND CLIENT_SECRET HAVE BEEN REMOVED DUE TO PRIVACY! 

In [54]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e05b8970cc1fd001b4946c8'},
 'response': {'headerLocation': "O'Connor - Parkview",
  'headerFullLocation': "O'Connor - Parkview, Toronto",
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.7108972045, 'lng': -79.30372360313615},
   'sw': {'lat': 43.701897195499996, 'lng': -79.31615039686386}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b5a3842f964a52023b528e3',
       'name': 'Jawny Bakers',
       'location': {'address': "804 O'Connor Dr",
        'crossStreet': 'St Clair E',
        'lat': 43.705782646822,
        'lng': -79.31291304477831,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.705782646822,
          'lng': -79.31291304477831}],
        'd

In [55]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [61]:
#data cleansing and structuring 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jawny Bakers,Gastropub,43.705783,-79.312913
1,East York Gymnastics,Gym / Fitness Center,43.710654,-79.309279
2,Shoppers Drug Mart,Pharmacy,43.705933,-79.312825
3,TD Canada Trust,Bank,43.705740,-79.312270
4,Pizza Pizza,Pizza Place,43.705159,-79.313130


In [62]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [63]:
#GETTING LOCATION FOR ALL NEIGHBOURHOODS IN EASTYORK
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
eastyork_venues = getNearbyVenues(names=EastYork_data['Neighborhood'],
                                   latitudes=EastYork_data['Latitude'],
                                   longitudes=EastYork_data['Longitude']
                                  )

Woodbine Gardens,Parkview Hill
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto


In [65]:
print(eastyork_venues.shape)
eastyork_venues.head()

(72, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
1,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
2,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,Shoppers Drug Mart,43.705933,-79.312825,Pharmacy
3,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank
4,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,Pizza Pizza,43.705159,-79.313130,Pizza Place


In [66]:
eastyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
East Toronto,2,2,2,2,2,2
Leaside,33,33,33,33,33,33
Thorncliffe Park,17,17,17,17,17,17
"Woodbine Gardens,Parkview Hill",10,10,10,10,10,10
Woodbine Heights,10,10,10,10,10,10


In [67]:
#Finding the unique categories
print('There are {} uniques categories.'.format(len(eastyork_venues['Venue Category'].unique())))

There are 46 uniques categories.


In [70]:
# Analysis of individual neighbourhoods
# one hot encoding
eastyork_onehot = pd.get_dummies(eastyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eastyork_onehot['Neighborhood'] = eastyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [eastyork_onehot.columns[-1]] + list(eastyork_onehot.columns[:-1])
eastyork_onehot = eastyork_onehot[fixed_columns]

eastyork_onehot.head()

,Neighborhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Housing Development,Indian Restaurant,Intersection,Liquor Store,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Video Store,Warehouse Store,Yoga Studio
0,"Woodbine Gardens,Parkview Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Woodbine Gardens,Parkview Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Woodbine Gardens,Parkview Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Woodbine Gardens,Parkview Hill",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Woodbine Gardens,Parkview Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
#group according to the mean frequency of occurrence of each category
eastyork_grouped = eastyork_onehot.groupby('Neighborhood').mean().reset_index()
eastyork_grouped

,Neighborhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Housing Development,Indian Restaurant,Intersection,Liquor Store,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Video Store,Warehouse Store,Yoga Studio
0,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,Leaside,0.0,0.030303,0.030303,0.030303,0.030303,0.030303,0.030303,0.060606,0.030303,0.121212,0.0,0.0,0.0,0.0,0.030303,0.030303,0.0,0.030303,0.060606,0.000000,0.0,0.030303,0.030303,0.0,0.000000,0.000000,0.0,0.030303,0.030303,0.000000,0.030303,0.000000,0.000000,0.030303,0.030303,0.030303,0.0,0.030303,0.0,0.090909,0.030303,0.030303,0.030303,0.0,0.000000,0.000000
2,Thorncliffe Park,0.0,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.058824,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.058824,0.058824,0.0,0.058824,0.117647,0.0,0.000000,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.058824,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.058824,0.000000,0.0,0.058824,0.058824
3,"Woodbine Gardens,Parkview Hill",0.1,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.1,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.100000,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,Woodbine Heights,0.1,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.1,0.1,0.1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000


## USING K_MEANS FOR CLUSTERING NEIGHBORHOODS

In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [80]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = eastyork_grouped['Neighborhood']

for ind in np.arange(eastyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eastyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,Convenience Store,Park,Cosmetics Shop,Gas Station,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Dessert Shop,Dance Studio
1,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Gym,Breakfast Spot,Fish & Chips Shop,Grocery Store,Clothing Store,Liquor Store
2,Thorncliffe Park,Burger Joint,Indian Restaurant,Yoga Studio,Pizza Place,Bank,Coffee Shop,Gas Station,Grocery Store,Warehouse Store,Housing Development
3,"Woodbine Gardens,Parkview Hill",Fast Food Restaurant,Pizza Place,Athletics & Sports,Pharmacy,Bank,Gym / Fitness Center,Intersection,Gastropub,Furniture / Home Store,Fish & Chips Shop
4,Woodbine Heights,Athletics & Sports,Video Store,Beer Store,Spa,Skating Rink,Dance Studio,Curling Ice,Cosmetics Shop,Park,Pharmacy


In [81]:
# set number of clusters to 5 as 5 neighborhoods
kclusters = 5

eastyork_grouped_clustering = eastyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eastyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 4, 3, 0])

In [82]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

eastyork_merged = EastYork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eastyork_merged  = eastyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

eastyork_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,3,Fast Food Restaurant,Pizza Place,Athletics & Sports,Pharmacy,Bank,Gym / Fitness Center,Intersection,Gastropub,Furniture / Home Store,Fish & Chips Shop
1,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,Athletics & Sports,Video Store,Beer Store,Spa,Skating Rink,Dance Studio,Curling Ice,Cosmetics Shop,Park,Pharmacy
2,M4G,East York,Leaside,43.709060,-79.363452,1,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Gym,Breakfast Spot,Fish & Chips Shop,Grocery Store,Clothing Store,Liquor Store
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372,4,Burger Joint,Indian Restaurant,Yoga Studio,Pizza Place,Bank,Coffee Shop,Gas Station,Grocery Store,Warehouse Store,Housing Development
4,M4J,East York,East Toronto,43.685347,-79.338106,2,Convenience Store,Park,Cosmetics Shop,Gas Station,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Dessert Shop,Dance Studio


## Visualising the clusters

In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eastyork_merged['Latitude'], eastyork_merged['Longitude'], eastyork_merged['Neighborhood'], eastyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## EXAMINING THE CLUSTERS

In [99]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 0, eastyork_merged.columns[[2] + list(range(5, eastyork_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Woodbine Heights,0,Athletics & Sports,Video Store,Beer Store,Spa,Skating Rink,Dance Studio,Curling Ice,Cosmetics Shop,Park,Pharmacy


In [100]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 1, eastyork_merged.columns[[2] + list(range(5, eastyork_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Leaside,1,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Gym,Breakfast Spot,Fish & Chips Shop,Grocery Store,Clothing Store,Liquor Store


In [101]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 2, eastyork_merged.columns[[2] + list(range(5, eastyork_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,2,Convenience Store,Park,Cosmetics Shop,Gas Station,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Dessert Shop,Dance Studio


In [102]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 3, eastyork_merged.columns[[2] + list(range(5, eastyork_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Woodbine Gardens,Parkview Hill",3,Fast Food Restaurant,Pizza Place,Athletics & Sports,Pharmacy,Bank,Gym / Fitness Center,Intersection,Gastropub,Furniture / Home Store,Fish & Chips Shop


In [103]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 4, eastyork_merged.columns[[2] + list(range(5, eastyork_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Thorncliffe Park,4,Burger Joint,Indian Restaurant,Yoga Studio,Pizza Place,Bank,Coffee Shop,Gas Station,Grocery Store,Warehouse Store,Housing Development
